## OBB 와 무게중심으로 정렬

In [6]:
import open3d as o3d
import numpy as np

# 1. 메쉬 로드
mesh = o3d.io.read_triangle_mesh("assets/data/ios_with_smilearch.stl")
mesh.compute_vertex_normals()
mesh.paint_uniform_color([0.7, 0.7, 0.7])

# 2. OBB와 중심점들 계산
obb = mesh.get_oriented_bounding_box()
R = np.asarray(obb.R)  # OBB의 rotation matrix
obb_center = obb.center    # OBB의 중심
mesh_center = mesh.get_center()  # 무게중심

# 중심점 차이 벡터 계산
center_diff = mesh_center - obb_center
print("중심점 차이 벡터:", center_diff)

# OBB 좌표계에서의 차이 벡터 (R의 각 축에 대한 프로젝션)
diff_in_obb = np.array([
    np.dot(center_diff, R[:, 0]),  # x축 방향 차이
    np.dot(center_diff, R[:, 1]),  # y축 방향 차이
    np.dot(center_diff, R[:, 2])   # z축 방향 차이
])
print("\nOBB 좌표계에서의 차이:", diff_in_obb)

# 3. 좌표축 생성
max_size = np.max(np.asarray(mesh.vertices).max(axis=0) - np.asarray(mesh.vertices).min(axis=0))
coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(
    size=max_size * 0.2,
    origin=obb_center
)

# 4. OBB 방향을 보여주는 선 생성
lines = []
colors = []
points = []
for i in range(3):
    direction = R[:, i] * max_size * 0.5
    points.append(obb_center - direction)
    points.append(obb_center + direction)
    colors.append([1, 0, 0] if i == 0 else [0, 1, 0] if i == 1 else [0, 0, 1])
    colors.append([1, 0, 0] if i == 0 else [0, 1, 0] if i == 1 else [0, 0, 1])

line_set = o3d.geometry.LineSet()
line_set.points = o3d.utility.Vector3dVector(points)
line_set.lines = o3d.utility.Vector2iVector([[2*i, 2*i+1] for i in range(3)])
line_set.colors = o3d.utility.Vector3dVector(colors)

# 중심점들 표시
obb_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=max_size * 0.02)
obb_sphere.translate(obb_center)
obb_sphere.paint_uniform_color([1, 0, 0])  # 빨간색 (OBB 중심)

mesh_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=max_size * 0.02)
mesh_sphere.translate(mesh_center)
mesh_sphere.paint_uniform_color([0, 0, 1])  # 파란색 (무게중심)

# 중심점 연결선 생성
center_line = o3d.geometry.LineSet()
center_line.points = o3d.utility.Vector3dVector([obb_center, mesh_center])
center_line.lines = o3d.utility.Vector2iVector([[0, 1]])
center_line.colors = o3d.utility.Vector3dVector([[1, 1, 0]])  # 노란색

# OBB 자체도 시각화
obb.color = [1, 0, 0]

# OBB의 정보 출력
print("\nOBB Rotation Matrix:")
print(R)
print("\nOBB Extent (크기):")
print(obb.extent)

# 시각화
o3d.visualization.draw_geometries(
    [mesh, coordinate_frame, line_set, obb, obb_sphere, mesh_sphere, center_line],
    mesh_show_back_face=True
)

중심점 차이 벡터: [-1.9539131  -0.71074882  2.26975509]

OBB 좌표계에서의 차이: [-1.98870442 -2.08695134 -1.07908167]

OBB Rotation Matrix:
[[ 0.99380274 -0.05935965  0.09398167]
 [-0.09850379 -0.07850832  0.992035  ]
 [-0.05150851 -0.99514465 -0.08386893]]

OBB Extent (크기):
[65.11596301 53.82793024 20.16435498]


In [ ]:
import open3d as o3d
import numpy as np
from collections import deque

# 1. 메쉬 로드 및 전처리
mesh = o3d.io.read_triangle_mesh("assets/data/ios_with_smilearch.stl")
print("\n=== 원본 메쉬 정보 ===")
print(f"원본 점 개수: {len(np.asarray(mesh.vertices))}")
print(f"원본 삼각형 개수: {len(np.asarray(mesh.triangles))}")

# 중복 정점 제거
mesh.remove_duplicated_vertices()
mesh.remove_duplicated_triangles()
mesh.remove_degenerate_triangles()
mesh.compute_vertex_normals()

print("\n=== 전처리 후 메쉬 정보 ===")
print(f"정리된 점 개수: {len(np.asarray(mesh.vertices))}")
print(f"정리된 삼각형 개수: {len(np.asarray(mesh.triangles))}")

# 2. 메쉬의 무게중심과 점들
mesh_center = mesh.get_center()
vertices = np.asarray(mesh.vertices)
vertex_normals = np.asarray(mesh.vertex_normals)
triangles = np.asarray(mesh.triangles)

# 3. 각 점에서 무게중심까지의 벡터
vectors = vertices - mesh_center
vectors_normalized = vectors / np.linalg.norm(vectors, axis=1)[:, np.newaxis]

# 4. 초기 seed 점들 선택 (z축 기준 각도)
angles_x = np.degrees(np.arctan2(vectors_normalized[:, 0], vectors_normalized[:, 2]))
angles_y = np.degrees(np.arctan2(vectors_normalized[:, 1], vectors_normalized[:, 2]))
initial_mask = (np.abs(angles_x) <= 40) & (np.abs(angles_y) <= 5)

# 시드 포인트 디버그
seed_indices = np.where(initial_mask)[0]
print("\n=== 시드 포인트 디버그 ===")
print(f"총 시드 포인트 수: {len(seed_indices)}")

# 시드 포인트 위치 중복 체크
seed_positions = vertices[seed_indices]
unique_positions = np.unique(seed_positions, axis=0)
print(f"고유한 시드 포인트 위치 수: {len(unique_positions)}")

# 5. 연결성 그래프 생성
def build_vertex_graph(triangles, vertices):
    graph = [set() for _ in range(len(vertices))]
    
    # 삼각형의 각 edge를 통해 연결성 구성
    for triangle in triangles:
        # 각 삼각형의 세 점을 서로 연결
        graph[triangle[0]].update([triangle[1], triangle[2]])
        graph[triangle[1]].update([triangle[0], triangle[2]])
        graph[triangle[2]].update([triangle[0], triangle[1]])
    
    # 디버그: 처음 몇 개의 연결 정보 출력
    print("\n=== 연결성 샘플 ===")
    for i in range(5):
        print(f"정점 {i}의 연결: {len(graph[i])}개 이웃")
    
    # set을 list로 변환
    return [list(neighbors) for neighbors in graph]

# 그래프 생성 및 디버그
print("\n=== 그래프 생성 시작 ===")
vertex_graph = build_vertex_graph(triangles, vertices)

# 연결성 통계
connections = [len(neighbors) for neighbors in vertex_graph]
print("\n=== 연결성 통계 ===")
print(f"평균 연결 수: {np.mean(connections):.2f}")
print(f"최소 연결 수: {np.min(connections)}")
print(f"최대 연결 수: {np.max(connections)}")
print(f"연결 없는 점의 수: {sum(1 for c in connections if c == 0)}")

# 6. Region Growing
def region_growing(vertices, normals, graph, initial_mask, angle_threshold=30):
    region = np.zeros(len(vertices), dtype=bool)
    region[initial_mask] = True
    
    # 시작점들의 평균 법선 계산
    seed_indices = np.where(initial_mask)[0]
    
    # BFS 큐 초기화
    queue = deque(seed_indices)
    visited = set(seed_indices)
    
    print("\n=== Region Growing 진행 상황 ===")
    print(f"시작점 개수: {len(seed_indices)}")
    
    iteration = 0
    last_visited_count = len(visited)
    
    while queue:
        current = queue.popleft()
        current_normal = normals[current]
        
        # 현재 점의 이웃들의 평균 법선 계산
        neighbor_normals = [normals[n] for n in graph[current] if region[n]]
        if not neighbor_normals:  # 이웃 중 영역에 포함된 점이 없으면 현재 법선 사용
            local_mean_normal = current_normal
        else:
            local_mean_normal = np.mean(neighbor_normals, axis=0)
            local_mean_normal = local_mean_normal / np.linalg.norm(local_mean_normal)
        
        # 이웃 점들 확인
        for neighbor in graph[current]:
            if neighbor in visited:
                continue
                
            neighbor_normal = normals[neighbor]
            
            # 이웃 점의 이웃들의 평균 법선 계산
            neighbor_neighbor_normals = [normals[n] for n in graph[neighbor]]
            neighbor_mean_normal = np.mean(neighbor_neighbor_normals, axis=0)
            neighbor_mean_normal = neighbor_mean_normal / np.linalg.norm(neighbor_mean_normal)
            
            # 1. 로컬 평균 법선과의 각도
            angle1 = np.degrees(np.arccos(np.clip(
                np.dot(local_mean_normal, neighbor_normal), -1.0, 1.0)))
            
            # 2. 이웃의 로컬 평균 법선과 현재 영역의 법선 각도
            angle2 = np.degrees(np.arccos(np.clip(
                np.dot(neighbor_mean_normal, local_mean_normal), -1.0, 1.0)))
            
            # 두 각도 조건 모두 만족해야 함
            if angle1 < angle_threshold and angle2 < angle_threshold:
                region[neighbor] = True
                queue.append(neighbor)
                visited.add(neighbor)
        
        # 진행 상황 출력
        if len(visited) - last_visited_count >= 1000:
            print(f"반복 {iteration}: 처리된 점 {len(visited)}개 (큐 크기: {len(queue)})")
            last_visited_count = len(visited)
            iteration += 1
            
        if iteration % 1000 == 0 and iteration > 0:
            print(f"현재까지 추가된 점 수: {len(visited) - len(seed_indices)}")
    
    return region

# Region Growing 실행
print("\n=== Region Growing 시작 ===")
grown_region = region_growing(
    vertices, 
    vertex_normals, 
    vertex_graph, 
    initial_mask,
    angle_threshold=8
)
# 7. 결과 시각화를 위한 색상 설정
colors = np.ones((len(vertices), 3)) * [0.7, 0.7, 0.7]  # 기본 회색
colors[initial_mask] = [0, 1, 0]  # 초기 seed 점들은 초록색
colors[grown_region & ~initial_mask] = [0, 0, 1]  # growing된 점들은 파란색
mesh.vertex_colors = o3d.utility.Vector3dVector(colors)

# 8. 좌표축 생성
max_size = np.max(vertices.max(axis=0) - vertices.min(axis=0))
coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(
    size=max_size * 0.2,
    origin=mesh_center
)

# 9. 무게중심 표시
center_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=max_size * 0.02)
center_sphere.translate(mesh_center)
center_sphere.paint_uniform_color([1, 0, 0])

# 10. 최종 결과 출력
print(f"\n=== 최종 결과 ===")
print(f"초기 seed 점들의 개수: {np.sum(initial_mask)}")
print(f"Region growing 후 점들의 개수: {np.sum(grown_region)}")
print(f"새로 추가된 점들의 개수: {np.sum(grown_region & ~initial_mask)}")

# 11. 시각화
o3d.visualization.draw_geometries(
    [mesh, coordinate_frame, center_sphere],
    mesh_show_back_face=True
)


=== 원본 메쉬 정보 ===
원본 점 개수: 492809
원본 삼각형 개수: 164284

=== 전처리 후 메쉬 정보 ===
정리된 점 개수: 82376
정리된 삼각형 개수: 164284

=== 시드 포인트 디버그 ===
총 시드 포인트 수: 1788
고유한 시드 포인트 위치 수: 1788

=== 그래프 생성 시작 ===

=== 연결성 샘플 ===
정점 0의 연결: 7개 이웃
정점 1의 연결: 7개 이웃
정점 2의 연결: 5개 이웃
정점 3의 연결: 6개 이웃
정점 4의 연결: 6개 이웃

=== 연결성 통계 ===
평균 연결 수: 5.99
최소 연결 수: 2
최대 연결 수: 12
연결 없는 점의 수: 0

=== Region Growing 시작 ===

=== Region Growing 진행 상황 ===
시작점 개수: 1788
